In [142]:
from selenium import webdriver

In [143]:
#imports
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as ec
from collections import deque
from selenium.webdriver.common.keys import Keys
import time


browser = webdriver.Chrome()
browser.get('https://web.telegram.org/#/im')

In [145]:
#global objects

# Home Group
HOME_GROUP = 'PAID GROUP'

# GroupQueue
groupQueue = deque([])


#grabs all groupchats
groupChats = browser.find_elements_by_class_name("im_dialog_wrap")

#Selected Top Group
selectedGroup = groupChats[0]

#Last Messagege Dictionary
lastMessageDict = {}

#Message Stack
MESSAGE_STACK = []

#Selected group title
SELECTED_GROUP_TITLE = ''


for groupChat in groupChats:
    key_gc = groupChat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
    lastMessageDict[key_gc] = ''

In [146]:
#groupQueueBuilder
def groupQueueBuilder():
    groupChats = browser.find_elements_by_class_name("im_dialog_wrap")
    badges = []
    titles = []

    for groupChat in groupChats:
        oneBadgeArray = groupChat.find_elements_by_css_selector("span.im_dialog_badge.badge")
        if(oneBadgeArray[0].text!=""):
            #get index and use this index in titlesArray
            oneTitleArray = groupChat.find_elements_by_css_selector("div.im_dialog_peer")
            if(oneTitleArray[0].text not in groupQueue):
                groupQueue.append(oneTitleArray[0].text)
    print(groupQueue)
groupQueueBuilder()

deque(['FGCU CRYPTO CRIPS', 'Telegram', 'Whale Group', ':palm_tree:Gulf Coast:palm_tree: Signals', 'Jason Cockinserter'])


In [147]:
#groupFinder - 
def groupFinder(name):
    groupQueue.popleft()
    chats = browser.find_elements_by_class_name("im_dialog_wrap")
    #print(chats)
    for chat in chats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == name):
            chat.click()
            selectedGroup = chat
            break;
    
groupFinder(groupQueue[0])
print(groupQueue)

deque(['Telegram', 'Whale Group', ':palm_tree:Gulf Coast:palm_tree: Signals', 'Jason Cockinserter'])


In [148]:
#Create push method to click on 'home chat' and paste and send it there.
# checks via dictionary which messages to push
def createMessageStack():
    MESSAGE_STACK.clear()
    messages = browser.find_elements_by_class_name("im_message_text")
    textMessages = []
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    #reversing list to make FIFO
    textMessages.reverse()
    
    SELECTED_GROUP_TITLE = selectedGroup.find_elements_by_css_selector("div.im_dialog_peer")[0].text
    print(lastMessageDict[SELECTED_GROUP_TITLE])
    for message in textMessages:
        if(message != lastMessageDict[SELECTED_GROUP_TITLE] and message != ''):
            MESSAGE_STACK.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            
            break
    print('message Stack:')
    print(MESSAGE_STACK)
        

createMessageStack()



message Stack:
['Ive heard of VEN.', 'well relatively high', 'it was rated highly in the weiss ratings thats why', 'Steem is blowing up', 'LTC usually follows BTC in terms of growth', 'Thoughts on LTC future? Thinking about buying some to hold for the future growth...if it ever happens lol', 'Looks like we might be starting to see a little stability and sideways motion', 'I saw some tweets saying 12-13$ after but idk how realistic that is', 'Anyone think anything will come out of Vens thing today at 2', 'Shit made me laugh', 'Is CMD a good buy now?', 'Have you heard about Ven?', 'Lol my dad randomly says to me, what’s XLM?', 'signal cV still strong i made a lot on it from pre sale but they have strong aspects!', 'Hey buy cV', '@stronghandz', 'https://www.reddit.com/r/CryptoCurrency/comments/7sl0fp/cnbc_thats_what_the_crypto_kids_called_getting/?st=JCTANK9I&sh=9088fe6b', '@JustinBattlett  Yea i got rid of the trojan smh', '@Rawbraw sounds like you got hit with the clipboard trojan..hav

In [152]:
# Broadcast Method
def broadCast():
    #select home chat
    print('Will now be sending:')
    print(MESSAGE_STACK)
    chats = browser.find_elements_by_class_name("im_dialog_wrap")
    for chat in chats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == HOME_GROUP):
            chat.click()
            break;
    
    #once homechat is selected, send messages one by one.
    textBox = browser.find_elements_by_class_name('composer_rich_textarea')[0]
    for message in MESSAGE_STACK:        
        textBox.send_keys(titleArray[0].text + ": " + message)
        textBox.send_keys(Keys.RETURN)
    
    lastMessageDict[SELECTED_GROUP_TITLE] = MESSAGE_STACK[-1]
    #clean up after sending messages.
    MESSAGE_STACK.clear()
    SELECTED_GROUP_TITLE= ''

print(SELECTED_GROUP_TITLE)   
broadCast()


Will now be sending:
['Ive heard of VEN.', 'well relatively high', 'it was rated highly in the weiss ratings thats why', 'Steem is blowing up', 'LTC usually follows BTC in terms of growth', 'Thoughts on LTC future? Thinking about buying some to hold for the future growth...if it ever happens lol', 'Looks like we might be starting to see a little stability and sideways motion', 'I saw some tweets saying 12-13$ after but idk how realistic that is', 'Anyone think anything will come out of Vens thing today at 2', 'Shit made me laugh', 'Is CMD a good buy now?', 'Have you heard about Ven?', 'Lol my dad randomly says to me, what’s XLM?', 'signal cV still strong i made a lot on it from pre sale but they have strong aspects!', 'Hey buy cV', '@stronghandz', 'https://www.reddit.com/r/CryptoCurrency/comments/7sl0fp/cnbc_thats_what_the_crypto_kids_called_getting/?st=JCTANK9I&sh=9088fe6b', '@JustinBattlett  Yea i got rid of the trojan smh', '@Rawbraw sounds like you got hit with the clipboard troja

UnboundLocalError: local variable 'SELECTED_GROUP_TITLE' referenced before assignment

In [13]:
#Bot Loop
while(true):
    if(len(list(groupQueue)) == 0):
        #timeout for x second(s)
        time.sleep(3)
    else:
        print('it is not 0')
        
            #get first in queue and cache / print all messages from it.
            #For our purpose, batch work scheduler would be better. **prioritizes active groupchats**

it is 0


In [14]:
def getMessagesFromGroup(groupName)

SyntaxError: invalid syntax (<ipython-input-14-a710443c59f5>, line 1)